In [1]:
import os
import cv2
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

In [2]:
dataset_path = r'C:\Users\Dell\Desktop\MANI\ndp\fer-2013'
 

In [3]:
emotions = ['angry', 'disgust', 'fear', 'happy', 'sad', 'surprise', 'neutral']


In [4]:
def load_images_from_folder(dataset_path):
    images = []
    labels = []
    
    for filename in os.listdir(dataset_path):
        if filename.endswith(".jpg"):
            emotion = filename.split('_')[0]  # Extract emotion from the filename
            if emotion in emotions:  # Check if the emotion is in the predefined list
                img_path = os.path.join(dataset_path, filename)
                img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)  # Read image in grayscale
                img = cv2.resize(img, (48, 48))  # Resize to 48x48
                img = img.astype('float32') / 255.0  # Normalize the image to [0, 1]
                
                images.append(img)
                labels.append(emotion)
    
    images = np.array(images)
    labels = np.array(labels)
    return images, labels

In [5]:
images, labels = load_images_from_folder(dataset_path)


In [6]:
label_encoder = LabelEncoder()
labels_encoded = label_encoder.fit_transform(labels)

# One-hot encode the labels
labels_one_hot = to_categorical(labels_encoded, num_classes=len(emotions))

# Expand dimensions of images to match the expected input shape (48, 48, 1)
images = np.expand_dims(images, axis=-1)


In [7]:
X_train, X_val, y_train, y_val = train_test_split(images, labels_one_hot, test_size=0.2, random_state=42)


In [8]:
def build_emotion_model():
    model = Sequential()
    model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(48, 48, 1)))
    model.add(MaxPooling2D((2, 2)))
    model.add(Conv2D(64, (3, 3), activation='relu'))
    model.add(MaxPooling2D((2, 2)))
    model.add(Conv2D(128, (3, 3), activation='relu'))
    model.add(MaxPooling2D((2, 2)))
    model.add(Flatten())
    model.add(Dense(128, activation='relu'))
    model.add(Dense(len(emotions), activation='softmax'))  # 7 emotions as output
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    return model

In [9]:
emotion_model = build_emotion_model()


C:\Users\Dell\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [10]:
emotion_model.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_val, y_val))


Epoch 1/10
692/692 ━━━━━━━━━━━━━━━━━━━━ 22s 29ms/step - accuracy: 0.2802 - loss: 1.7704 - val_accuracy: 0.3989 - val_loss: 1.5329
Epoch 2/10
692/692 ━━━━━━━━━━━━━━━━━━━━ 21s 30ms/step - accuracy: 0.4449 - loss: 1.4536 - val_accuracy: 0.4851 - val_loss: 1.3456
Epoch 3/10
692/692 ━━━━━━━━━━━━━━━━━━━━ 20s 29ms/step - accuracy: 0.5097 - loss: 1.2945 - val_accuracy: 0.4927 - val_loss: 1.3219
Epoch 4/10
692/692 ━━━━━━━━━━━━━━━━━━━━ 20s 29ms/step - accuracy: 0.5524 - loss: 1.1968 - val_accuracy: 0.5268 - val_loss: 1.2511
Epoch 5/10
692/692 ━━━━━━━━━━━━━━━━━━━━ 20s 29ms/step - accuracy: 0.5841 - loss: 1.1077 - val_accuracy: 0.5335 - val_loss: 1.2283
Epoch 6/10
692/692 ━━━━━━━━━━━━━━━━━━━━ 21s 30ms/step - accuracy: 0.6024 - loss: 1.0578 - val_accuracy: 0.5342 - val_loss: 1.2249
Epoch 7/10
692/692 ━━━━━━━━━━━━━━━━━━━━ 20s 29ms/step - accuracy: 0.6352 - loss: 0.9802 - val_accuracy: 0.5346 - val_loss: 1.2586
Epoch 8/10
692/692 ━━━━━━━━━━━━━━━━━━━━ 20s 29ms/step - accuracy: 0.6681 - loss: 0.8936 - 

In [11]:
loss, accuracy = emotion_model.evaluate(X_val, y_val)
print(f"Validation Accuracy: {accuracy*100:.2f}%")


173/173 ━━━━━━━━━━━━━━━━━━━━ 2s 9ms/step - accuracy: 0.5496 - loss: 1.3001
Validation Accuracy: 54.40%


In [12]:
emotion_model.save('trainingmodels/models/emotion_detection_model.h5')

In [24]:
def predict_emotion_from_image(model, image_path):
    # Load the image in grayscale
    img = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
    
    # Check if the image was loaded successfully
    if img is None:
        print(f"Error: Unable to load image at {image_path}")
        return None  # Return None or an error message
    
    # Resize the image to 48x48 pixels
    img = cv2.resize(img, (48, 48))
    
    # Normalize the image to [0, 1]
    img = img.astype('float32') / 255.0
    
    # Expand dimensions to match the model input shape: (1, 48, 48, 1)
    img = np.expand_dims(img, axis=-1)  # Add channel dimension
    img = np.expand_dims(img, axis=0)  # Add batch dimension
    
    # Predict the emotion
    predictions = model.predict(img)
    
    # Get the predicted class index
    predicted_class = np.argmax(predictions)
    
    # Map the predicted class index back to the emotion label
    emotion = label_encoder.inverse_transform([predicted_class])[0]
    
    return emotion

# Example usage to predict emotion from an image


In [30]:
test_image_path = r'C:\Users\Dell\Downloads\th (1).jpeg'  # Replace with your image path
predicted_emotion = predict_emotion_from_image(emotion_model, test_image_path)
print(f"The predicted emotion is: {predicted_emotion}")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step
The predicted emotion is: happy
